# Generate MACD using daily data service - F#

### Overview
*Moving Average Convergence/Divergence Oscillator* was developed by Gerald Appel and represents the difference between two moving averages of different lengths.

Commonly used MACD the 12-day EMA less the 26-day EMA, but the parameters can be adapted when required. As its name implies, MACD is all about convergence and divergence of the two moving averages

MACD oscilaltes around the zero line as follows:
* positive values increase as the shorter moving average diverges further from the longer one
* negative values indicate that the shorter moving average is below the longer one

This sample demonstrates how to request and plot **on-demand** *Moving Average Convergence/Divergence Oscillator - MACD* from a daily data service.

### Inputs/outputs
Moving Average Convergence/Divergence Oscillator's request require instrument's identifier, date time intervals and number of periods of each moving average as per inputs.

*MACD* is computed as follows:

$$ MACD = 12day,EMA - 26day,EMA $$

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve MACD indicator from the hosted service. The queried endpoint in this script are:
* *DailyMacdService*: to directly retrieve daily data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily MACD sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.DailyAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data

#### 3.1 Request Parameters
To request *daily MACD* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The MACD parameters

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 MACD parameters

In [ ]:
// Set EMAs lengths
let longMacd = 26
let shortMacd = 12

#### 3.3 Time period selection

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2019, Month = 01, Day = 01 ),
    EndDate = new Date ( Year = 2020, Month = 12, Day = 31 )
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)

#### 3.4 MACD request creation
The following code snippet creates *gRPC client*, process daily MACD requests and returns the reply.

In [ ]:
// Instantiate the daily MACD service
let service = new DailyMacdService.DailyMacdServiceClient(channel)

// Create and process the daily MACD request
let request = 
    new DailyMacdRequest( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ), 
        Short = shortMacd, 
        Long = longMacd,
        Constraints = constraints
    )
let reply = service.DailyMacd(request, headers)

// Visualize requests' results
//display(dailyMacdReply.Data)

### Step 4: Visualize data

#### 4.1 Select data

In [ ]:
// Extract the price and the macd
let prices    = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value)))
let macdData  = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Macd)))
let longMacd  = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Long)))
let shortMacd = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Short)))

#### 4.2 Plot MACD and daily prices
Plot MACD request results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
let macdGraph = 
    [
        new Scattergl(
            name = "MACD",
            mode = "lines",
            x = (macdData |> Seq.map fst),
            y = (macdData |> Seq.map snd),
            yaxis = "y2"
        )
        new Scattergl(
            name = "Price",
            mode = "lines",
            x = (prices |> Seq.map fst),
            y = (prices |> Seq.map snd)
        )
        new Scattergl(
            name = "Long EMA",
            mode = "lines",
            x = (longMacd |> Seq.map fst),
            y = (longMacd |> Seq.map snd)
        )
        new Scattergl(
            name = "Short EMA",
            mode = "lines",
            x = (shortMacd |> Seq.map fst),
            y = (shortMacd |> Seq.map snd)
        )
    ]

let layout = 
    new Layout.Layout( 
        title = $"MACD and daily prices for {ticker}-{exchange}",
        xaxis = new Xaxis (title = "Date"),
        yaxis = new Yaxis (
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "MACD", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(macdGraph, layout)
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display